In [3]:
import scanpy as sc
import anndata
import logging
import os
import numpy as np
import pandas as pd
import json
# import scanpy.external as sce
logging.basicConfig(level=logging.INFO)
import scvi
import torch

from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
import seaborn as sns

# from PySpots.fish_helpers import *
from dredFISH.Analysis import basicu

import importlib
importlib.reload(basicu)
sc.set_figure_params(figsize=(7, 7))

In [ ]:
logging.info('hi')

# file paths and load data

In [ ]:
def get_mse(y_true, y_pred):
    """
    """
    mse = np.power(y_true-y_pred, 2).mean()
    return mse

def get_r2(y_true, y_pred):
    """
    """
    # r2 = 1-(np.power(y_true-y_pred, 2).mean()/np.power(y_true-np.mean(y_true, axis=0), 2).mean())
    r2 = 1 - np.linalg.norm(y_true-y_pred)**2/np.linalg.norm(y_true-np.mean(y_true, axis=0))**2
    return r2

In [ ]:
prj_dir = '/bigstore/GeneralStorage/fangming/projects/dredfish/'
dat_dir = prj_dir + 'data/'
res_dir = prj_dir + 'data_dump/'

# allen data
# scrna_path = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_DPNMF_matrix.h5ad'
scrna_path = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_DPNMF_genes_matrix.h5ad'

# analysis metadata
meta_path = '/bigstore/GeneralStorage/fangming/projects/dredfish/data_dump/analysis_meta_Mar31.json'

In [ ]:
# allen scrna matrix
ref_data = anndata.read_h5ad(scrna_path)

# analysis
with open(meta_path, 'r') as fh:
    meta = json.load(fh)
ref_data

In [ ]:
ref_data.obs = ref_data.obs.rename({
                                    'class_label': 'Level_1_class_label',
                                    'neighborhood_label': 'Level_2_neighborhood_label',
                                    'subclass_label': 'Level_3_subclass_label',
                                    'cluster_label': 'Level_5_cluster_label',
                                    }, axis=1)
ref_data

In [ ]:
adata = ref_data.copy()
X = adata.X.todense().astype(int) # within the support of Poisson
adata.X = X 
adata

In [9]:
# split and test
np.random.seed(0)
n_latents = [10, 20, 50, 128]
for n_latent in n_latents: 
    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True)
    for train_idx, test_idx in kf.split(X):
        adata_train = adata[train_idx].copy()
        adata_test = adata[test_idx].copy()

        # model
        scvi.model.SCVI.setup_anndata(adata_train, layer=None, batch_key=None)
        vae = scvi.model.SCVI(adata_train, 
                              n_hidden=128,
                              n_latent=n_latent, 
                              n_layers=2, 
                              gene_likelihood="poisson")

        # train
        # vae.train(max_epochs=3)
        vae.train(max_epochs=3)

        # test
        # z = vae.get_latent_representation()
        # rho_scvi = vae.get_normalized_expression() # fraction of gene expression for each cell (sums to 1)
        # Xhat = vae.posterior_predictive_sample()
        Xhat_test = vae.posterior_predictive_sample(adata=adata_test)

        # eval
        X_test = adata_test.X
        Xn_test = np.log10(X_test+1)
        Xhatn_test = np.log10(Xhat_test+1)
        mse = get_mse(Xn_test, Xhatn_test) 
        r2 = get_r2(Xn_test, Xhatn_test)

        print(f"n_latent={n_latent}, MSE={mse}, r2={r2}")
        
        # save
        output_model_dir = f'/bigstore/GeneralStorage/fangming/projects/dredfish/data_dump/scvi_model_nz{n_latent}_May24'
        prefix = ''
        vae.save(output_model_dir, prefix, overwrite=True, save_anndata=True)
        break

INFO:absl:Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
INFO:absl:Unable to initialize backend 'gpu': NOT_FOUND: Could not find registered platform with name: "cuda". Available platform names are: Host Interpreter
INFO:absl:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpap9a8yfn
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpap9a8yfn/_remote_module_non_sriptable.py
/home/fangming/anaconda3/envs/scvi_v3p9/lib/python3.9/site-packages/torch/cuda/__init__.py:82: UserWarning:

CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755849709/work/c10/cuda/CUDAFunc

Epoch 218/218: 100%|██████████| 218/218 [1:31:50<00:00, 25.28s/it, loss=9.19e+04, v_num=1]
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup       
MSE=0.41996644892913415, r2=-0.07499887230838032
CPU times: user 1d 23h 35min 50s, sys: 1h 35min 7s, total: 2d 1h 10min 57s
Wall time: 1h 33min 22s


In [11]:
# # test reload
# vae2 = scvi.model.SCVI.load(dir_path=output_model_dir, prefix=prefix) # need adata?
# vae2

INFO     File /bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scvi_model_halfdat
         a_April21/model.pt already downloaded                                               


SCVI Model with the following params: 
n_hidden: 128, n_latent: 10, n_layers: 2, dropout_rate: 0.1, dispersion: gene, 
gene_likelihood: poisson, latent_distribution: normal
Training status: Trained